In [1]:
# Load the appropriate libraries
shh <-suppressPackageStartupMessages
install.packages('ggcorrplot')
install.packages('pROC')
install.packages('randomForest')
shh(library("randomForest"))
shh(library(ggplot2))
shh(library(caret))
shh(library(ggcorrplot))
shh(library(pROC))

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


# HR Attrition Analysis
In this analysis, I will attempt to explore the data given, try to understand the factors of attrition and build a model to decipher it. By doing so, I hope to be able to find ways to reduce attrition and improve the financial results of the company. The results will be based on observable data and therefore cannot be used alone to distinguish reasons for attrition. However, it can be useful for preventive measures and also to increase the rate of retention.

In [2]:
#Load the CSV file with employee data.
hr_data <- read.csv("https://bit.ly/2N2ggiK")

In [3]:
#Calculate the dimension of the dataset.
dim(hr_data)

[1] 1470   25

In [4]:
#Preview the first few records in the dataset.
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


In [5]:
# List and describe each of the fields in the dataset.
str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : int  1 0 1 0 0 0 0 0 0 0 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 ...
 $ DistanceFromHome   

In [6]:
#Calculate descriptive stats for each of the fields in the dataset.
summary(hr_data)

   Attrition      EmployeeNumber        Age           Gender     MaritalStatus
 Min.   :0.0000   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 1st Qu.:0.0000   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
 Median :0.0000   Median :1020.5   Median :36.00                Single  :470  
 Mean   :0.1612   Mean   :1024.9   Mean   :36.92                              
 3rd Qu.:0.0000   3rd Qu.:1555.8   3rd Qu.:43.00                              
 Max.   :1.0000   Max.   :2068.0   Max.   :60.00                              
                                                                              
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :

In [7]:
cor <- cor(hr_data[c(1,3,8,12:25)])
options(repr.plot.width=7, repr.plot.height=5)
ggcorrplot(cor)

## GGPlots that shows the factors of attrition##
The following are graphs that will aid the process of identifying factors of attrition. These graphs are plotted based on several elements that influence the workforce.

#### 1."Does one gender tend to quit more frequently than the other?"

In [30]:
ggplot(hr_data,aes(x=Gender , fill = factor(Attrition))) + 
    geom_bar(position= "fill") + xlab("Gender") + ylab("Attrition") + 
 scale_fill_manual(values = c("grey", "turquoise"), name = "Did they Quit?", labels = c("No", "Yes"))

#### 2."Does age appear to make a difference? Are our younger workers less committed to our company?"

In [9]:
ggplot(hr_data,aes(x=Age, fill = factor(Attrition))) + 
    geom_density(position= "fill") + scale_fill_manual(values = c("grey", "lightblue"), name = "Did they Quit?", labels = c("No", "Yes"))

#### 3."Are we working our employees too hard? Is our overtime requirement for some employees driving them away?"

In [10]:
ggplot(hr_data,aes(OverTime,fill=factor(Attrition))) + 
  geom_bar() + xlab("Overtime") + ylab("Attrition") + 
  scale_fill_manual(values = c("grey", "lightpink"), name = "Did they Quit?", labels = c("No", "Yes"))

#### 4."Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?"

In [11]:
ggplot(hr_data,aes(Attrition,PercentSalaryHike)) + geom_jitter()+ ylab("Percent Salary Hike")+
  xlab("Attrition") + (aes(Department))

#### 5."Our concern is that the highest performing employees are leaving at a higher rate. Is that true?"

In [12]:
ggplot(hr_data,aes(x=PerformanceRating, fill = factor(Attrition))) + 
    geom_density(position= "fill") +
    scale_fill_manual(values = c("grey", "lightgreen"), name = "Did they Quit?", labels = c("No", "Yes"))

## Additional Graphs

#### Does their marital status affect attrition?

In [33]:
ggplot(hr_data,aes(x=MaritalStatus , fill = factor(Attrition))) + 
    geom_bar(position= "fill") + 
    scale_fill_manual(values = c("grey", "red"), name = "Did they Quit?", labels = c("No", "Yes"))

In [14]:
#Determine if any missing/null records exists
sum(is.na(hr_data))
na.omit(hr_data)

[1] 0

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6
0,10,59,Female,Married,Research & Development,Laboratory Technician,2670,Yes,Medical,...,4,20,3,3,12,4,1,0,0,0
0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,4,22,1,2,1,1,1,0,0,0
0,12,38,Male,Single,Research & Development,Manufacturing Director,9526,No,Life Sciences,...,4,21,0,2,10,0,9,7,1,8


In [15]:
hr_model_dt <- randomForest(Attrition~ Age + Gender + MonthlyIncome + OverTime + Department,
   method="class", data=hr_data)

plot(hr_model_dt)

Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

In [16]:
# Splits the data into training and test sets 
set.seed(3456)
trainIndex <- createDataPartition(hr_data$Attrition, p=.8, 
                                     list=FALSE, times=1)
hr_train <- hr_data[ trainIndex,]
hr_test  <- hr_data[-trainIndex,]

In [17]:
#  Use glm to create your prediction model.
hr_model <- glm (Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
                          data=hr_train, family=binomial)
summary(hr_model)


Call:
glm(formula = Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
    family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1450  -0.6253  -0.4929  -0.2987   2.7028  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -7.442e-01  4.282e-01  -1.738 0.082244 .  
Age                  -3.137e-02  1.098e-02  -2.856 0.004294 ** 
GenderMale            3.057e-01  1.746e-01   1.750 0.080050 .  
MaritalStatusMarried  3.146e-01  2.501e-01   1.258 0.208392    
MaritalStatusSingle   1.029e+00  2.481e-01   4.148 3.35e-05 ***
MonthlyIncome        -9.775e-05  2.768e-05  -3.531 0.000414 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1023.56  on 1175  degrees of freedom
Residual deviance:  942.48  on 1170  degrees of freedom
AIC: 954.48

Number of Fisher Scoring iterations: 5


In [18]:
# Assess the importance of each variable.
varImp(hr_model)

,Overall
,<dbl>
Age,2.855730
GenderMale,1.750394
MaritalStatusMarried,1.257999
MaritalStatusSingle,4.148028
MonthlyIncome,3.531139


In [19]:
# Add prediction in the test dataset.
hr_test$Attrition_prediction <- predict(hr_model, newdata=hr_test,type='response')
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_prediction
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,13,0,2,8,0,7,7,3,6,0.3289514
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,22,1,2,1,1,1,0,0,0,0.1620813
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,13,1,5,6,0,5,4,0,3,0.1886208
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,12,2,5,7,0,6,2,0,5,0.1462027
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,23,0,4,10,7,5,3,0,3,0.2949338
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,11,0,4,13,0,12,6,2,11,0.1241285


In [20]:
ROC <- roc(hr_test$Attrition, hr_test$Attrition_prediction)

plot(ROC)
auc(ROC)

Setting levels: control = 0, case = 1
Setting direction: controls < cases


Area under the curve: 0.6564

In [21]:
# Write the "Attrition" preductions to a new file.
hr_test_abbreviated <- hr_test[,c(1,2,26)]
head(hr_test_abbreviated)
summary(hr_test_abbreviated)
write.csv(hr_test_abbreviated,"hr_predictions(1).csv",row.names=FALSE)

,Attrition,EmployeeNumber,Attrition_prediction
,<int>,<int>,<dbl>
6,0,8,0.3289514
8,0,11,0.1620813
11,0,14,0.1886208
17,0,21,0.1462027
22,1,27,0.2949338
23,0,28,0.1241285


   Attrition      EmployeeNumber   Attrition_prediction
 Min.   :0.0000   Min.   :   8.0   Min.   :0.01154     
 1st Qu.:0.0000   1st Qu.: 489.5   1st Qu.:0.08824     
 Median :0.0000   Median :1023.0   Median :0.14616     
 Mean   :0.1769   Mean   :1008.9   Mean   :0.15888     
 3rd Qu.:0.0000   3rd Qu.:1507.8   3rd Qu.:0.21401     
 Max.   :1.0000   Max.   :2044.0   Max.   :0.45304     

## Observations

Based on observations, there is no one main reason that employees quit. Pay raises are only part of the solution. People need to feel appreciated. They need opportunities for growth. This of course cannot be collected as data. This study suggests that in order to reduce attrition, industries should create some opportunities for the growth of their employees within the organization by adopting new innovative technologies and effective training programs.

Resource: https://pdfs.semanticscholar.org/ec3e/6b5834ac020aa9e6eebca5fb814083ca5bd8.pdf

### Recommendations
After proper analysis, I have concluded that the attrition of employees is high between the age group of 30-40. Hence, I suggest that 
benefits and or incentives be given to these individuals to decrease the attrition level and thereby reducing cost. It is more costly to attract and retrain new individuals. A large number of them works in the R&D department which can greatly affect the productivity of the company. 